# Machine Learning Phase Transitions in the 2D Ising Model
### Amal Sebastian

This notebook presents a machine learning–based analysis of the 2D Ising model. The project explores how convolutional neural networks (CNNs) can identify the ordered and disordered phases of the Ising model, learn temperature from raw spin configurations, and estimate the critical temperature.

---

## 1. Background and Motivation

The 2D Ising model is a classical model in statistical mechanics describing spins on a lattice with nearest-neighbor interactions. It is described by the simple Hamiltonian:

$$
H = -J \sum_{\langle i,j \rangle} s_i s_j
$$

Here, the $s_i = \{-1, 1\}$ are  binary magnetic moments (spins) sitting on a two-dimensional square lattice, and $\langle i,j \rangle$ indicates that only interactions between neighboring spins are taken into account. For simplicity, we will set \(J = 1\). At low temperatures, the system is ordered (almost all spins are aligned), but if we raise the temperature above a critical temperature Tc, the temperature fluctuations destroy the order and there is no preferred spin alignment any more (compare Fig. 1). Therefore, there exists a phase transition and the critical temperature $T_c$ at which the transiition occurs was calculated by Lars Onsager in 1944 to be :

$$
T_c = \frac{2}{\log(1 + \sqrt{2})} = 2.269
$$

Below $T_c$, spins tend to align → ordered phase.  
Above $T_c$, thermal fluctuations dominate → disordered phase.

### **Research Questions**
1. Can a CNN learn to classify Ising configurations into ordered vs. disordered phases?
2. Can a CNN regress the temperature directly from the spin configuration?
3. Can CNN be used to estimate the critical temperature at which such a transition occurs?


Machine learning offers a powerful route to detect phase transitions *without explicitly computing magnetization or other order parameters*. This demonstrates how ML extracts physical structure from raw many-body configurations.


![description](pictures/lattice_3d.png)
---

## 2. Methodology

The project was divided into five major components:

1. **Data Generation** using Metropolis–Hastings Monte Carlo.
2. **Data Preprocessing** (normalization, labels, train/val/test splits).
3. **CNN Phase Classification** (ordered vs disordered).
4. **CNN Temperature Regression** (predict continuous T).
5. **Critical Temperature Estimation** 

---

### 2.1 Data Generation: Metropolis–Hastings Algorithm

The 2D Ising model was simulated on a square lattice of size 32×32. For each temperature T, the system was equilibrated using Metropolis updates, and a fixed number of spin configurations were collected.

- Temperatures used:  1 - 5 in steps of 0.4

- Number of configurations per temperature: 500

The data were saved as .npy files, each containing all configurations at a given temperature. Below we visualize a few configurations at low, near-critical, and high temperatures. The black means spin up and white means spin down.


![description](pictures/visualize_lattice.png)

### 2.2 Data Preprocessing

The preprocessing script was used to achieve the following:

- Normalized spins from {-1,1} to {0,1}
- Assigned phase labels based on the temperature ( for classification). 0 is ordered while 1 is disordered
- Split data into training,validation and test sets for both regression and classification,
    * 70% training  
    * 15% validation  
    * 15% testing  

All datasets are saved into `.npz` files.


### 2.3. CNN Phase Classification

The model needs to be trained for phase classification . For that we use a Convolutional Neural Network (CNN). CNN is a deep learning architecture designed specifically for processing image data. It automatically learns hierarchical features directly from raw pixels using convolutional layers that detect local patterns such as edges, textures, and shapes. As the network goes deeper, these features combine to form higher-level representations of the image . We use the following architecture with CNN:

- Conv2D(32) + ReLU + MaxPool
- Conv2D(64) + ReLU + MaxPool
- Conv2D(128) + ReLU
- Flatten → Dense(128) → Dropout(0.2)
- Output: Sigmoid (binary classification)

Loss: Binary Cross-Entropy  
Optimizer: Adam  
Epochs: 300  
Checkpointing: Best validation loss

CNN was used here because it  learns local correlations and domain structures typical of the two phases. The loss function is given below whose trend means that the model learned well.

![description](pictures/loss_class.png)


### 2.4 Temperature Regression

To regress temperature, we reuse a similar CNN architecture as above but change the final layer to linear activation.

Loss: Mean-Squared Error  
Optimizer: Adam  
Epochs: 300  
Checkpointing: Best validation loss

This task is more challenging because the mapping between domain structure and temperature is continuous and nonlinear. Following is the loss-curve for the task and we can say that it learnt well from the trend .


![description](pictures/loss_reg.png)

---


## 3. Results

### 3.1 Phase Classification

 CNN achieved high accuracy on both training and validation sets. The confusion mattrix showed that the it could clearly identify between the two phases.

 ![description](pictures/confusion_mattrix.png)



### 3.2 Temperature Regression

The regression model successfully learned to predict temperatures. At low temperatures when there is some order, the predictions are so much better compared to higher temperatures where the spins are all randomly aligned. This is shown in the following figure.

![description](pictures/reg_temp.png)

### 3.3 Finding Critical Temperature

The prediction of phase and temperature was both made on the same test set and they were plotted against each other as shown in the figure. We see that the phase is starting to change around 2.2-2.5 temperature units. This region bound the actual physics calculated value of 2.269 units as mentioned in the introduction. So the CNN was able to identify the Transition region reasonably well.

![description](pictures/critical_temp.png)

---

## 4. Synthesis and Discussion


CNNs successfully classified phases of the Ising model with high accuracy. CNNs also regressed temperatures from raw spin configurations. The learned critical temperature of 2.2-2.5 matches physics expectations reasonably well.

### 4.1 Obstacles encountered

- Regression becomes unstable near the critical region.
- Some temperature predictions deviate due to strong fluctuations.
- Overfitting was a risk due to small dataset sizes at individual temperatures.
- Data preprocessing required careful balancing between classes.
- The hyper parameter learning rate took time to adjust.

### 4.2 What we would improve
- Use more temperature points to capture the transition well especially near transition temperature.
- Try a more expressive architecture (ResNet-like small CNN).
- Use unsupervised learning methods (PCA, t-SNE on features).
- Train a classifier without labels to detect the transition.

### 4.3 Final Conclusion

This project demonstrates that machine learning, even with relatively simple CNNs, can:
- learn phase structure directly from configurations
- perform physical regression tasks
- identify critical behavior without using the magnetization or any analytic order parameter